In [ ]:
import sys
print(sys.version)

In [ ]:
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1 0 n1
s space 1 n1 n2
pd pout n2
xaxis laser P lin 1 10 100
"""
kat.parse(code)
out = kat.run()
out.plot()

In [ ]:
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l l1 1 0 n0
s s1 1 n0 n1
m m1 0.5 0.5 0 n1 n2
pd pd1 n2

xaxis l1 P lin 0 10 10
"""

kat.parse(code)
out = kat.run()
out.plot()

* `l l1 1 0 n0`
    * `l` laser
    * `l1` name of component
    * `1` P (power in Watts)
    * `0` frequency offset
    * no phase (optional)
    * `n0` connected node
* `s s1 1 n0 n1`
    * `s` free space
    * `s1` component name
    * `1` length (in meters)
    * optional index of refraction (default 1)
    * `n0` `n1` connected nodes
* `m m1 0.5 0.5 0 n1 n2`
    * `m` mirror
    * `m1` component name
    * `0.5` R (reflectivity)
    * `0.5` T (power transmission)
    * `0` phi (tuning in degrees)
    * `n1 n2` connected nodes
* `pd pd1 n2`
    * `pd` photo-diode
    * `pd1` component name
    * `n2` connected node
* `xaxis l1 P lin 0 10 10`
    * x-axis definition (parameter to tune)
    * `l1` component to tune
    * `P` parameter of component to tune
    * `lin` linear
    * `0` from (Watts)
    * `10` to (Watts)
    * steps